(Written by Amir Akhavan)

In [11]:
import requests
import pandas as pd
import json
from pathlib import Path

In [2]:
PREPROCESSED_DATA_PATH = Path('../.../data/preprocessed/property_assessor')

In [3]:
county_fips_code = '45039'

In [4]:
initial  = "https://beacon.schneidercorp.com/Application.aspx?AppID=796&LayerID=16296"
x_initial = requests.get(initial)
x_initial = x_initial.content.decode("utf-8")

In [5]:
qps_location = x_initial.find('QPS')
qps_closing_str = x_initial[qps_location+6:]
end = qps_closing_str.find('"')
QPS = qps_closing_str[:end]

In [13]:
QPS

'F9tZvOIY_XZOdUskmaGw54VoDEVPL0bY8E9muKXf1q0Y3RtiKO-fXkr2WvfxscDkbVAKnYudfhVepVVn-oXYgP-dqgsX8get-bxXhSkh4h4NzFwuFmgkFApZxPKkUjTUOnZS8m05u_47GWkMiLArf-BELR2u4O_IsifbxqlkvNtpyWSI9NEdFPBGUG59ko3ZnbGXYYaGVTaUEWlA9LDwawoz8bXObWcDd2mFX2k6SDnSaEkr5mgwe-AqziPV5IUfvXBMc1qOAcgs1In8v_8iYw2'

In [80]:
url = f'https://qpublic.schneidercorp.com/api/beaconCore/BufferAndExport?QPS={QPS}'
myobj = {
    'layerId': '16297', #  original: "11834",
    'keys': ["001-00-00-003-000", "150-00-00-004-000"],
    'distance': 100, # 00000,
    'exportType': "xlsx",
    # 'keys': ["001-00-00-003-000", "028-00-00-004-000"],
    'showParcelId': 'true',
    'skipLabelCount': "0",
    'usePropertyAddress': 'true',
    # my additions:
    #'Visible': 'true',
    #'LayerName': 'Parcels',
    #'Query': 'true',
    #"HasMap":'true',
}
x = requests.post(url, data = myobj)
url_of_csv="https://beacon.schneidercorp.com/"

In [81]:
x.content

b'{"d":{"DownloadUrl":"/Download.aspx?FileName=FairfieldCountySC-20200706-8deb47d932d248d4b93d03f492265d4f.xlsx","InputFeatureCount":1,"OutputFeatureCount":5,"ElapsedMilliseconds":192,"BufferGeometry":"MULTIPOLYGON (((1876104.6420288335 974298.38862328581, 1876104.6899218585 974317.14525449579, 1876106.4957458552 974335.80522251432, 1876109.0597053671 974350.59848137025, 1876127.1921853777 974422.58491343074, 1876128.5875989038 974427.57512998162, 1876156.5468949366 974518.4177540649, 1876158.4484383946 974524.00414720492, 1876170.1422624027 974555.29937923269, 1876173.8379094757 974563.93048568617, 1876224.9514276162 974669.33375604276, 1876227.7966293658 974677.06370765879, 1876230.9019127272 974693.7837935529, 1876231.7023008626 974721.00064994011, 1876230.956353178 974729.235613122, 1876216.435102311 974735.72233507584, 1876198.1138220865 974746.36500806431, 1876187.9142700715 974753.83905606787, 1876173.9739968127 974766.20704564592, 1876164.9087808079 974775.90330165764, 1876154.

In [82]:
str_vals = json.loads(x.content)
download_url = url_of_csv+str_vals['d']['DownloadUrl']
df = pd.read_excel(download_url)
df = df.drop_duplicates()

In [83]:
str_vals['d']

{'DownloadUrl': '/Download.aspx?FileName=FairfieldCountySC-20200706-8deb47d932d248d4b93d03f492265d4f.xlsx',
 'InputFeatureCount': 1,
 'OutputFeatureCount': 5,
 'ElapsedMilliseconds': 192,
 'BufferGeometry': 'MULTIPOLYGON (((1876104.6420288335 974298.38862328581, 1876104.6899218585 974317.14525449579, 1876106.4957458552 974335.80522251432, 1876109.0597053671 974350.59848137025, 1876127.1921853777 974422.58491343074, 1876128.5875989038 974427.57512998162, 1876156.5468949366 974518.4177540649, 1876158.4484383946 974524.00414720492, 1876170.1422624027 974555.29937923269, 1876173.8379094757 974563.93048568617, 1876224.9514276162 974669.33375604276, 1876227.7966293658 974677.06370765879, 1876230.9019127272 974693.7837935529, 1876231.7023008626 974721.00064994011, 1876230.956353178 974729.235613122, 1876216.435102311 974735.72233507584, 1876198.1138220865 974746.36500806431, 1876187.9142700715 974753.83905606787, 1876173.9739968127 974766.20704564592, 1876164.9087808079 974775.90330165764, 18

In [74]:
geometry = str_vals['d']['BufferGeometry'].split('MULTIPOLYGON')[1].replace('(', '[').replace(')', ']')

In [84]:
df

,ParcelId,OwnerName,Address,City,State,Zip
0,028-00-00-008-000,CURRENT RESIDENT,161 BLUEFIELD DRIVE,Fairfield,SC,NaN
1,028-00-00-014-000,CURRENT RESIDENT,114 BLUEFIELD DRIVE,Fairfield,SC,NaN
2,028-00-00-036-000,CURRENT RESIDENT,MCCONNELL RD TRL,Fairfield,SC,NaN
3,028-00-00-011-000,CURRENT RESIDENT,752 MCCONNELL ROAD,Fairfield,SC,NaN
4,028-00-00-009-000,CURRENT RESIDENT,101 BLUEFIELD DRIVE,Fairfield,SC,NaN


In [77]:
df = df.set_index('ParcelId')
df

,OwnerName,Address,City,State,Zip
ParcelId,,,,,
028-00-00-008-000,CURRENT RESIDENT,161 BLUEFIELD DRIVE,Fairfield,SC,NaN
028-00-00-014-000,CURRENT RESIDENT,114 BLUEFIELD DRIVE,Fairfield,SC,NaN
028-00-00-036-000,CURRENT RESIDENT,MCCONNELL RD TRL,Fairfield,SC,NaN
028-00-00-011-000,CURRENT RESIDENT,752 MCCONNELL ROAD,Fairfield,SC,NaN
028-00-00-009-000,CURRENT RESIDENT,101 BLUEFIELD DRIVE,Fairfield,SC,NaN


In [86]:
parcel_ids = df['ParcelId']
parcel_ids

0    028-00-00-008-000
1    028-00-00-014-000
2    028-00-00-036-000
3    028-00-00-011-000
4    028-00-00-009-000
Name: ParcelId, dtype: object

In [ ]:
property_lookup_url = 'https://beacon.schneidercorp.com/Application.aspx?AppID=796&LayerID=11834&PageTypeID=4&PageID=5738&Q=1649139588&KeyValue='
for parcel_id in parcel_ids:
    url = property_lookup_url + parcel_id
    

In [8]:
path = PREPROCESSED_DATA_PATH / 'fips' / county_fips_code
!mkdir -p {path}
df.to_csv(path / 'properties.csv')

In [9]:
!wc -l {path}/properties.csv

17245 ../../data/preprocessed/property_assessor/fips/45039/properties.csv


In [10]:
df

,ParcelId,OwnerName,Address,City,State,Zip
0,216-00-00-005-000,CURRENT RESIDENT,2100FT W OF INT S-20-323 & S-20-324,Fairfield,SC,NaN
1,215-00-00-074-000,CURRENT RESIDENT,293 EMERALD DR TRACT,Fairfield,SC,NaN
2,215-00-00-102-000,CURRENT RESIDENT,285 EMERALD DR,Fairfield,SC,NaN
3,026-04-02-005-000,CURRENT RESIDENT,SW OF GIBSON END RD RD,Fairfield,SC,NaN
4,026-04-02-007-000,CURRENT RESIDENT,557 GIBSON END ROAD,Fairfield,SC,NaN
...,...,...,...,...,...,...
17480,141-00-00-031-000,CURRENT RESIDENT,S 2000FT INT S-20-115 & HWY 34 E H,Fairfield,SC,NaN
17481,077-01-00-002-000,CURRENT RESIDENT,1704 GREAT NORTH RD RD,Fairfield,SC,NaN
17482,048-00-00-002-000,CURRENT RESIDENT,"HWY 215 N, CLOWNEY & JOHN BRICE RDS",Fairfield,SC,NaN
17483,047-00-00-047-000,CURRENT RESIDENT,JOHN BRICE RD,Fairfield,SC,NaN
